In [1]:
%run preprocecssing_helpers.ipynb
import os
import time
import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams.update({'font.size': 15})
# matplotlib.rcParams.

In [2]:
# path = 'F:\StroopEEG'
path = 'F:\EEGLAB'

eeglab_files = os.listdir(path)
eeglab_dict = search_folders(path, eeglab_files)
eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
eeglab_files

['F:\\EEGLAB\\SA20131216\\20131216_1310.set',
 'F:\\EEGLAB\\SA20131216\\20131216_1441.set',
 'F:\\EEGLAB\\SA20140205\\20140205_1114.set',
 'F:\\EEGLAB\\SA20140205\\20140205_1230.set',
 'F:\\EEGLAB\\SA20140206\\20140206_1029.set',
 'F:\\EEGLAB\\SA20140206\\20140206_1151.set',
 'F:\\EEGLAB\\SA20140207\\20140207_1146.set',
 'F:\\EEGLAB\\SA20140207\\20140207_1219.set',
 'F:\\EEGLAB\\SA20140217\\20140217_1136.set',
 'F:\\EEGLAB\\SA20140217\\20140217_1211.set',
 'F:\\EEGLAB\\SA20140219\\20140219_1046.set',
 'F:\\EEGLAB\\SA20140219\\20140219_1205.set',
 'F:\\EEGLAB\\SA20140220\\20140220_1009.set',
 'F:\\EEGLAB\\SA20140220\\20140220_1123.set',
 'F:\\EEGLAB\\SA20140225\\20140225_1422.set',
 'F:\\EEGLAB\\SA20140225\\20140225_1552.set',
 'F:\\EEGLAB\\SA20140227A\\20140227_1058.set',
 'F:\\EEGLAB\\SA20140227A\\20140227_1220.set',
 'F:\\EEGLAB\\SA20140227B\\20140227_1410.set']

In [22]:
raw = mne.io.read_raw_eeglab(eeglab_files[2],preload = True)

Reading F:\EEGLAB\SA20140205\20140205_1114.fdt
Reading 0 ... 859270  =      0.000 ...  1678.262 secs...


In [12]:
def list_eeglab_files(root_path):
    """
    NOTE: must perform magic command %run preprocecssing_helpers.ipynb before using this method.
    
    Description: 
        Lists all eeglab .set files within a directory. 
        Assuming that the file format is: 
        Root Folder -> Subject Folder -> eeglab file contents
        example: 
        EEGLAB (root) -> SA20140220 (subject) -> 20140220_1009.set
        or 
        F:\\EEGLAB\\SA20140220\\20140220_1009.set
    
    Variables: 
        root_path: the folder containing all subject folders
    ----
    Returns:
        List of file .set path locations.
    """
    
    eeglab_files_list = os.listdir(root_path)
    eeglab_dict = search_folders(root_path, eeglab_files_list)
    eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
    
    return eeglab_files

In [ ]:
eeglab_files = list_eeglab_files(path)

In [17]:
def find_events(eeglab):
    """
    Description:
        returns the events of an eeglab .set object
        
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    ------
    returns: np array of 3d lists
    """
    return mne.find_events(raw, output= 'offset',
                              consecutive= True,
                              min_duration=0.001953125, 
                              shortest_event= 2, 
                              initial_event=False)
    

In [56]:
events = find_events(raw)
type(events)

5793 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]


numpy.ndarray

In [49]:
def get_trials(events):
    """
    Descriptions:
        returns a list of trials where each trial contains timestamp, offset and event_id
    
    Variables:
        events: A np array of 3d lists. use mne.find_events or custom find_events(eeglab) method already written.
    -----
    returns: np array of lists
    """
    trials = []
    current_trial = []
    for event in events:
        if event[-1] == 31 and current_trial != []:
            trials.append(current_trial)
            current_trial = []
            current_trial.append(event)
        else:
            current_trial.append(event)
            
    trials.append(current_trial) # the last trial does not have an indicator, so append it after the loop ends
    return trials

In [55]:
trials = get_trials(events)

In [60]:
trials[1][0]

31

In [129]:
def filter_events(eeglab):
    """
    Description:
        Filters out all events that are not within the 1-24 stim-code definitions.
        Also filters for trials that have a reaction time greater than .500ms as the subject 
        failed to click in time and was marked incorrect anyway.
        
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    ------
    returns:
        new_events: 
            np array of 3d list of events 
    """
    
    events = find_events(eeglab)
    trials = get_trials(events)
    button_clicks = [131,132]
    accuracy = [129,130]
    found_first_stim_code = False
    found_third_stim_code = False
    isEnd_of_trial = False
    new_events = []
    good_trials = []
    bad_trials_count = 0
    for trial in trials:
#         print(trial)
        for event in trial:
            # find first stim-code
            if event[-1] <= 24 and found_first_stim_code == False: 
                first_stim_code = event
                found_first_stim_code = True
#                 print("found first stim-code: {0}".format(first_stim_code))

            # find third stim-code
            if found_first_stim_code and event[-1] <= 24:
                #the third stim-code is one less than the first. 
                if event[-1] + 1 == first_stim_code[-1]:
                    third_stim_code = event
                    found_third_stim_code = True
            
            if event[-1] in button_clicks:
                button_click = event        
                
            if event[-1] in accuracy:
                #end of trial
                if event[-1] == 130: # incorrect
                    button_click = event
                    time_diff = (button_click[0] - third_stim_code[0]) * (1/512) #compute time in seconds
                    if time_diff > 1:
                        print("time difference {0}".format(time_diff))
                        print(third_stim_code)
                        print(button_click)
                        bad_trials_count+=1
                    else:
                        #store this trial
                         good_trials.append(trial)
                else:
                    good_trials.append(trial)
                found_first_stim_code = False
                found_third_stim_code = False
                
    print("\n **{0} bad trials dropped**".format(bad_trials_count))
    return np.concatenate([x for x in good_trials])

In [130]:
new_events = np.array(filter_events(eeglab = raw))

5793 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
time difference 1.060546875
[8655    0   13]
[9198    0  130]
time difference 1.4765625
[100574      0     16]
[101330      0    130]
time difference 1.109375
[303457      0      1]
[304025      0    130]
time difference 1.111328125
[679756      0     22]
[680325      0    130]

 **4 bad trials dropped**


In [32]:
tmax = (1/raw.info['sfreq']) * 499 # tmax 1/sfrq * desired epoch length
epoch = mne.Epochs(raw, events=new_events, event_id = new_stim_code_event_ids, preload = True, tmin=0, tmax=tmax) 

1920 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1920 events and 500 original time points ...
0 bad epochs dropped


In [5]:
event_id = {'trail_start':  31, 
            'left_button':  131, 
            'right_button': 132,
            'congruent':    133,
            'incongruent':  134,
            'correct':      129,
            'incorrect':    130,
            # targets
            't_baby' :        135,
            't_bell':         136,
            't_bird':         137,
            't_burp':         138,
            't_dog':          139,
            't_drum':         140,
            't_knock':        141,
            't_laugh':        142,
            't_phone':        143,
            't_train':        144,
            't_water':        145,
            #flankers
            'f_baby' :        155,
            'f_bell':         156,
            'f_bird':         157,
            'f_burp':         158,
            'f_dog':          159,
            'f_drum':         160,
            'f_knock':        161,
            'f_laugh':        162,
            'f_phone':        163,
            'f_train':        164,
            'f_water':        165,
}
# event codes 1 - 24 represent flanker and target stim codes. They are NOT explicity defined in dissertation.
for i in range(1,13):
    event_id["flanker_stim_{0}".format(i)] = i
    
for j in range(13, 25):
    event_id["target_stim_{0}".format(j)] = j

In [ ]:
# for set_file in eeglab_files[0]:
#     try:
#         start = time.time()
#         raw = mne.io.read_raw_eeglab(set_file, preload=True)
#         raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
#         raw.filter(5,100, fir_design='firwin')
#         raw.resample(256, n_jobs=10)
#         ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
#         ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
#         ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#                      ecg_score_func='pearsonr', skew_criterion=None, 
#                      kurt_criterion=None, var_criterion=None,add_nodes=None)
#         raw_ica = ica.apply(raw)

#         m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 1, initial_event=True)
#         epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)
# #         epoch.save(set_file.split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)
#         end = time.time()
#         print("SUCESSFULLY LOADED time elapsed: {0}", (end - start))
#     except :
#         print("***Error converting file: {0}***".format(set_file))
        

In [27]:
new_stim_code_event_ids = {
     
    "b-f2-wrd":1,
    "b-f1-wrd":2,
    "b-tg-wrd":3,

    "b-f2-spk":4,
    "b-f1-spk":5,
    "b-tg-spk":6,

    "b-f2-pic":7,
    "b-f1-pic":8,
    "b-tg-pic":9,

    "b-f2-snd":10,
    "b-f1-snd":11,
    "b-tg-snd":12,

    "f-f2-wrd":13,
    "f-f1-wrd":14,
    "f-tg-wrd":15,

    "f-f2-spk":16,
    "f-f1-spk":17,
    "f-tg-spk":18,

    "f-f2-pic":19,
    "f-f1-pic":20,
    "f-tg-pic":21,

    "f-f2-snd":22,
    "f-f1-snd":23,
    "f-tg-snd":24
}

In [3]:
# start = time.time()
# # raw = mne.io.read_raw_eeglab(, preload=True)
# raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
# raw.filter(5,100, fir_design='firwin')
# raw.resample(256, n_jobs=10)
# ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
# ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
# ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#              ecg_score_func='pearsonr', skew_criterion=None, 
#              kurt_criterion=None, var_criterion=None,add_nodes=None)
# raw_ica = ica.apply(raw)

# # m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 0.5, initial_event=True)
# epoch = mne.Epochs(raw, events=new_events, event_id =new_stim_code_event_ids , preload = True, tmin=0, tmax=1,)
# end = time.time()
# print("time elapsed: {0}", (end - start))

## check if saving and loading epoch file works.

In [ ]:
os.chdir(epoch_destination)

In [ ]:

epoch.save('20131216_1310_epo.fif',verbose = True)

In [ ]:
epoch_files = os.listdir(epoch_destination)

In [ ]:
epoch_file = epoch_files[0]
epoch_ = mne.read_epochs(epoch_file,preload=True)

In [ ]:
epoch_.plot(title="epochs");

In [ ]:
for set_file in eeglab_files:
    print(set_file.split('\\')[-1].split('.')[0] + "_epo.fif")

In [ ]:
# start = time.time()
raw = mne.io.read_raw_eeglab(eeglab_files[1], preload=True)
raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
raw.filter(5,100, fir_design='firwin')
raw.resample(256, n_jobs=10)
ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
             ecg_score_func='pearsonr', skew_criterion=None, 
             kurt_criterion=None, var_criterion=None,add_nodes=None)
raw_ica = ica.apply(raw)

m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 1.5, initial_event=True)
epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)

## Read epoched data any plot

In [ ]:
path = 'E:\EpochedEEG'
epoch_files = os.listdir(path)

In [ ]:
eeg_file = os.path.join(path,epoch_files[0])

In [ ]:
epoch = mne.read_epochs(eeg_file, preload = True)

In [ ]:
epoch.plot(n_channels=20, n_epochs = 10, title = eeg_file);